In [18]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time, csv
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.relative_locator import locate_with

In [351]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\panag\AppData\Local\Temp/ipykernel_19148/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [352]:
page_url = 'https://www.novibet.gr/en/sports'
driver.get(page_url)
cookies = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.CLASS_NAME, 'acceptCookies_button'))).click()

In [356]:
# Close Log in pop up window
x_button = driver.find_element(By.CSS_SELECTOR, '[data-cy="closeBtn"]').click()

## 1. Football

In [358]:
# Click Daily Coupon (Football)
dayly_coupon_button = driver.find_element(By.CSS_SELECTOR, 'a.ng-star-inserted[title="Daily coupon"]').click()

In [360]:
# Wait for the dailyCoupon_body element to be present on the page
wait = WebDriverWait(driver, 10)
# By deafult is the football first
daily_coupon_body = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'dailyCoupon_body')))

# Extract footbal text
football_string = daily_coupon_body.text

In [361]:
# Create list from the initial string
initial_list = football_string.split('\n')

# Remove first elements of the list not needed
remove_elements = ['Daily coupon','Football','Tennis','Basketball',
                   '24 hours','12 hours','3 hours','Popular First','SO']

football_list = [x for x in initial_list if x not in remove_elements]

### I want to find a way to increace the size of i if i insert elements

# Use padding for 'Markets are not available'
index = 0
while index < len(football_list):
    if football_list[index] == 'Markets are not available':
        football_list.insert(index+1, 'No_market')
        football_list.insert(index+2, 'No_market')
        football_list.insert(index+3, 'No_market')
        index += 4
    else:
        index += 1

In [362]:
football_string

'Daily coupon\nFootball\nBasketball\nTennis\n24 hours\n12 hours\n3 hours\nPopular First\nEurope - EURO 2024 - Qualifying\nKazakhstan\nSlovenia\nThu 17:00\n1\n4.10\nX\n3.30\n2\n1.98\nO 2.5\n2.35\nU 2.5\n1.59\nGG\n2.10\nNG\n1.71\nBosnia & Herzegovina\nIceland\nThu 21:45\n1\n1.84\nX\n3.65\n2\n4.30\nO 2.5\n2.20\nU 2.5\n1.69\nGG\n2.04\nNG\n1.75\nDenmark\nFinland\nThu 21:45\n1\n1.36\nX\n4.80\n2\n9.60\nO 2.5\n2.01\nU 2.5\n1.81\nGG\n2.50\nNG\n1.52\nItaly\nEngland\nSO\nThu 21:45\n1\n3.05\nX\n3.30\n2\n2.68\nO 2.5\n2.25\nU 2.5\n1.64\nGG\n1.93\nNG\n1.84\nNorth Macedonia\nMalta\nThu 21:45\n1\n1.38\nX\n4.80\n2\n8.70\nO 2.5\n1.88\nU 2.5\n1.93\nGG\n2.25\nNG\n1.62\nPortugal\nLiechtenstein\nThu 21:45\n1\n1.001\nX\n41.00\n2\n61.00\nO 6.5\n2.07\nU 6.5\n1.74\nGG\n4.40\nNG\n1.19\nSan Marino\nNorthern Ireland\nThu 21:45\n1\n51.00\nX\n9.60\n2\n1.07\nO 3.5\n2.10\nU 3.5\n1.70\nGG\n3.90\nNG\n1.23\nSlovakia\nLuxembourg\nThu 21:45\n1\n1.52\nX\n4.45\n2\n5.90\nO 2.5\n2.01\nU 2.5\n1.81\nGG\n2.15\nNG\n1.67\nAfrica - A

In [363]:
# Create sublists based on Championship
championship = [x for x in football_list if ' - ' in x]
index_championship = [i for i,x in enumerate(football_list) if ' - ' in x]
sublists_championships = [football_list[i:j] for i, j in zip([0]+index_championship, index_championship + [len(football_list)])]
# Exclude the initial empty list from sublist_championships
sublists_championships = sublists_championships[1:]

In [364]:
# Create dataframe 
df_football = pd.DataFrame(columns = ['championship','team1','team2','time','1','odds_1','X','odds_X','2','odds_2',
                             'Over','odds_over','Under','odds_under','GG','odds_gg','NG','odds_ng'])

# Loop through all championships
for j in range(len(sublists_championships)):
     
    team1_lst, team2_lst = [],[]
    time_lst = []
    one_lst, one_odds_lst, x_lst, x_odds_lst, two_lst, two_odds_lst = [],[],[],[],[],[]
    over_lst, over_odds_lst, under_lst, under_odds_lst = [],[],[],[]
    gg_lst, gg_odds_lst, ng_lst, ng_odds_lst = [],[],[],[]

    # Create list with only teams and odds / exclude the championship at the beginning
    teams_only_lst = sublists_championships[j][1:]
    championship_lst = [sublists_championships[j][0] for _ in range(len(teams_only_lst) // 17)]
    

    i = 0
    while i < len(teams_only_lst):
        team1_lst.append(teams_only_lst[i])
        team2_lst.append(teams_only_lst[i+1])
        time_lst.append(teams_only_lst[i+2])
        one_lst.append(teams_only_lst[i+3])
        one_odds_lst.append(teams_only_lst[i+4])
        x_lst.append(teams_only_lst[i+5])
        x_odds_lst.append(teams_only_lst[i+6])
        two_lst.append(teams_only_lst[i+7])
        two_odds_lst.append(teams_only_lst[i+8])
        over_lst.append(teams_only_lst[i+9])
        over_odds_lst.append(teams_only_lst[i+10])
        under_lst.append(teams_only_lst[i+11])
        under_odds_lst.append(teams_only_lst[i+12])
        gg_lst.append(teams_only_lst[i+13])
        gg_odds_lst.append(teams_only_lst[i+14])
        ng_lst.append(teams_only_lst[i+15])
        ng_odds_lst.append(teams_only_lst[i+16])
        i = i + 17

    # Create the dataframe
    df = pd.DataFrame({
        'championship': championship_lst,
        'team1': team1_lst,
        'team2': team2_lst,
        'time': time_lst,
        '1': one_lst,
        'odds_1': one_odds_lst,
        'X': x_lst,
        'odds_X': x_odds_lst,
        '2': two_lst,
        'odds_2': two_odds_lst,
        'Over': over_lst,
        'odds_over': over_odds_lst,
        'Under': under_lst,
        'odds_under': under_odds_lst,
        'GG': gg_lst,
        'odds_gg': gg_odds_lst,
        'NG': ng_lst,
        'odds_ng': ng_odds_lst
    })

    df_football = df_football.append(df, ignore_index = True)

In [365]:
df_football

,championship,team1,team2,time,1,odds_1,X,odds_X,2,odds_2,Over,odds_over,Under,odds_under,GG,odds_gg,NG,odds_ng
0,Europe - EURO 2024 - Qualifying,Kazakhstan,Slovenia,Thu 17:00,1,4.10,X,3.30,2,1.98,O 2.5,2.35,U 2.5,1.59,GG,2.10,NG,1.71
1,Europe - EURO 2024 - Qualifying,Bosnia & Herzegovina,Iceland,Thu 21:45,1,1.84,X,3.65,2,4.30,O 2.5,2.20,U 2.5,1.69,GG,2.04,NG,1.75
2,Europe - EURO 2024 - Qualifying,Denmark,Finland,Thu 21:45,1,1.36,X,4.80,2,9.60,O 2.5,2.01,U 2.5,1.81,GG,2.50,NG,1.52
3,Europe - EURO 2024 - Qualifying,Italy,England,Thu 21:45,1,3.05,X,3.30,2,2.68,O 2.5,2.25,U 2.5,1.64,GG,1.93,NG,1.84
4,Europe - EURO 2024 - Qualifying,North Macedonia,Malta,Thu 21:45,1,1.38,X,4.80,2,8.70,O 2.5,1.88,U 2.5,1.93,GG,2.25,NG,1.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Central America - CONCACAF Nations League,British Virgin Islands,Puerto Rico,Thu 21:00,1,41.00,X,13.00,2,1.01,O 5.5,2.09,U 5.5,1.69,GG,2.80,NG,1.40
76,Central America - CONCACAF Nations League,ST Martin,Saint Kitts and Nevis,Thu 21:00,1,4.30,X,4.00,2,1.70,O 3.5,2.30,U 3.5,1.59,GG,1.55,NG,2.35
77,Bahrain - Federations Cup,Hidd Club,Al Khalidiyah,Thu 20:45,1,5.60,X,3.75,2,1.55,O 2.5,1.93,U 2.5,1.82,GG,1.99,NG,1.76
78,Bahrain - Federations Cup,Manama Club,AL Bahrain,Thu 20:45,1,1.44,X,4.15,2,6.40,O 2.5,1.91,U 2.5,1.83,GG,2.10,NG,1.67


## 2. Basket

In [366]:
# Click the basketball button
basketball_button = driver.find_element(By.CSS_SELECTOR, '[class = "svgImage default BASKETBALL_GAME medium"]').click()

# Wait for the dailyCoupon_body element to be present on the page
wait = WebDriverWait(driver, 10)
daily_coupon_body = wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'dailyCoupon_body')))

# Extract basketball text
basketball_string = daily_coupon_body.text

In [368]:
# Create list from the initial string
initial_list = basketball_string.split('\n')

# Remove first elements of the list not needed
remove_elements = ['Daily coupon','Football','Tennis','Basketball',
                   '24 hours','12 hours','3 hours','Popular First','SO']

basketball_list = [x for x in initial_list if x not in remove_elements]

# Use padding for 'Markets are not available'
index = 0
while index < len(basketball_list):
    if basketball_list[index] == 'Markets are not available':
        basketball_list.insert(index+1, 'No_market')
        basketball_list.insert(index+2, 'No_market')
        basketball_list.insert(index+3, 'No_market')
        index += 4
    else:
        index += 1

In [370]:
# Create sublists based on Championship
championship = [x for x in basketball_list if ' - ' in x]
index_championship = [i for i,x in enumerate(basketball_list) if ' - ' in x]
sublists_championships = [basketball_list[i:j] for i, j in zip([0]+index_championship, index_championship + [len(basketball_list)])]
# Exclude the initial empty list from sublist_championships
sublists_championships = sublists_championships[1:]

In [ ]:
# Create dataframe 
df_basketball = pd.DataFrame(columns = ['championship','team1','team2','time','1','odds_1','X','odds_X','2','odds_2',
                             'Over','odds_over','Under','odds_under','GG','odds_gg','NG','odds_ng'])

# Loop through all championships
for j in range(len(sublists_championships)):
     
    team1_lst, team2_lst = [],[]
    time_lst = []
    one_lst, one_odds_lst, x_lst, x_odds_lst, two_lst, two_odds_lst = [],[],[],[],[],[]
    over_lst, over_odds_lst, under_lst, under_odds_lst = [],[],[],[]
    gg_lst, gg_odds_lst, ng_lst, ng_odds_lst = [],[],[],[]

    # Create list with only teams and odds / exclude the championship at the beginning
    teams_only_lst = sublists_championships[j][1:]
    championship_lst = [sublists_championships[j][0] for _ in range(len(teams_only_lst) // 17)]
    

    i = 0
    while i < len(teams_only_lst):
        team1_lst.append(teams_only_lst[i])
        team2_lst.append(teams_only_lst[i+1])
        time_lst.append(teams_only_lst[i+2])
        one_lst.append(teams_only_lst[i+3])
        one_odds_lst.append(teams_only_lst[i+4])
        x_lst.append(teams_only_lst[i+5])
        x_odds_lst.append(teams_only_lst[i+6])
        two_lst.append(teams_only_lst[i+7])
        two_odds_lst.append(teams_only_lst[i+8])
        over_lst.append(teams_only_lst[i+9])
        over_odds_lst.append(teams_only_lst[i+10])
        under_lst.append(teams_only_lst[i+11])
        under_odds_lst.append(teams_only_lst[i+12])
        gg_lst.append(teams_only_lst[i+13])
        gg_odds_lst.append(teams_only_lst[i+14])
        ng_lst.append(teams_only_lst[i+15])
        ng_odds_lst.append(teams_only_lst[i+16])
        i = i + 17

    # Create the dataframe
    df = pd.DataFrame({
        'championship': championship_lst,
        'team1': team1_lst,
        'team2': team2_lst,
        'time': time_lst,
        '1': one_lst,
        'odds_1': one_odds_lst,
        'X': x_lst,
        'odds_X': x_odds_lst,
        '2': two_lst,
        'odds_2': two_odds_lst,
        'Over': over_lst,
        'odds_over': over_odds_lst,
        'Under': under_lst,
        'odds_under': under_odds_lst,
        'GG': gg_lst,
        'odds_gg': gg_odds_lst,
        'NG': ng_lst,
        'odds_ng': ng_odds_lst
    })

    df_football = df_football.append(df, ignore_index = True)